In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler

data = pd.read_csv(r'C:\Users\Kerry\tweet_sentiment.csv')
print(data.shape)
data.head()

(1000, 2)


,tweet,sentiment
0,The event starts at 5 PM.,neutral
1,I hate how this turned out.,negative
2,Fantastic experience!,positive
3,Fantastic experience!,positive
4,This is the worst thing ever!,negative


In [2]:
data.isnull().sum()

tweet        0
sentiment    0
dtype: int64

In [3]:
data.duplicated().sum()

982

In [4]:
data = data.drop_duplicates()

In [5]:
data.describe()

,tweet,sentiment
count,18,18
unique,18,3
top,The event starts at 5 PM.,neutral
freq,1,6


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18 entries, 0 to 85
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet      18 non-null     object
 1   sentiment  18 non-null     object
dtypes: object(2)
memory usage: 432.0+ bytes


In [10]:
import re

def clean_tweet(tweet):
    tweet = re.sub(r"http\S+", "", tweet)  # remove URLs
    tweet = re.sub(r"@\w+", "", tweet)     # remove mentions
    tweet = re.sub(r"#\w+", "", tweet)     # remove hashtags
    tweet = re.sub(r"[^A-Za-z\s]", "", tweet)  # remove non-letters
    return tweet.lower()

data['clean_text'] = data['tweet'].astype(str).apply(clean_tweet)
data[['tweet', 'clean_text']].head()


,tweet,clean_text
0,The event starts at 5 PM.,the event starts at pm
1,I hate how this turned out.,i hate how this turned out
2,Fantastic experience!,fantastic experience
4,This is the worst thing ever!,this is the worst thing ever
6,What a wonderful day!,what a wonderful day
